In [1]:
import enum

class POS(enum.Enum):
    """
    This class is an enumerated type containing syntactic attributes of words and phrases and can be
    used to build grammars
    
    Attributes
    See individual comments
    
    Methods
    N/A
    """
    
    #types
    T = 0 #Tense
    N = 1 #Noun
    V = 2 #Verb
    Adj = 3 #Adjective
    Adv = 4 #Adverb
    P = 5 #Preposition
    Det = 6 #Determiner
    #levels
    Bar = 7 #Bar'
    Phrase = 8 #Phrase
    #add more specific information
    Transitive = 9#Verbs
    Intransitive = 10
    Ditransitive = 13 #Not sure how to implement this in a binary tree at the moment...
    Proper = 11#Nouns
    Pro = 12
    Mass = 14
    Count = 15
    Plural = 16


In [2]:
class Constituent:
    """
    This class represents a tree of syntactic constituents
    
    Attributes
    word: str
        the word contained in this Constituent, if this is a phrasal head / leaf node, None otherwise
    
    left: Constituent
        the left child node, if this is a non-terminal node with two children
        or the only child of a node with a single child
    
    right: Constituent
        the right child node, if this is a non-terminal node with two children
        or the only child of a node with a single child
        
    pos: POS tuple
        the part of speech of this Constituent
        
    Methods
    formatPos(): str
        a helper method that formats the pos attribute for use in __repr__() and __str__()
    """
    
    def __init__(self, word = None, left = None, right = None, pos = None):
        """
        Constructs all neccesary attributes for the Constituent object
        
        Parameters
        word : str, optional
            the word contained in this Constituent, if this is a phrasal head / leaf node, None otherwise

        left : Constituent, optional
            the left child node, if this is a non-terminal node with two children
            or the only child of a node with a single child
            
        right : Constituent, optional
            the right child node, if this is a non-terminal node with two children
            or the only child of a node with a single child
            
        pos : POS tuple
            the part of speech of this Constituent
        """
        self.word = word
        self.left = left
        self.right = right
        if isinstance(pos, tuple):
            self.pos = pos
        else:
            self.pos = (pos,)
            
    def __repr__(self):
        if self.word != None:
            return "[" + self.formatPos() + " " + self.word + "]"
        if self.right == self.left:
            return "[" + self.formatPos() + " " + str(self.left) + "]"
        return "[" + self.formatPos() + " " + str(self.left) + " " + str(self.right) + "]"
    
    def __str__(self):
        return self.__repr__()
    
    def formatPos(self):
        """
        A helper method that formats the pos attribute for use in __repr__() and __str__()
        
        Parameters
        N/A
        
        Returns
        None
        """
        s = str()
        for p in self.pos:
            if p == POS.N:
                s = s + "N"
            elif p == POS.V:
                s = s + "V"
            elif p == POS.T:
                s = s + "T"
            elif p == POS.Det:
                s = s + "Det"
            elif p == POS.P:
                s = s + "P"
            elif p == POS.Adj:
                s = s + "Adj"
            elif p == POS.Adv:
                s = s + "Adv"
            if p == POS.Phrase:
                s = s + "P"
            elif p == POS.Bar:
                s = s + "'"
        return s

In [3]:
import re
class Grammar:
    """
    A class representing a grammar in memory, including the lexicon, unary production rules, and binary production rules
    
    Attributes
    uniGrammar: dict
        a dictionary using POS objects to represent unary production rules
    binGrammar: dict
        a dictionary using POS objects to represent binary production rules
    lexicon: dict
        a dictionary using POS objects and strings to represent word categories
        
    Methods
    add(i, o): 
        simplified way to enter grammar rules, sorts grammar rules by content to use addBin(), addUni(), or addLexicon()
    addBin(constituents, phrase):
        adds a binary production rule to self.binGrammar in which the constituents join together to form a phrase
    addUni(constituent, phrase):
        adds a unary production rule to self.uniGrammar in which the constituent is also a phrase
    addLexicon(word, pos):
        adds an entry to the lexicon in which the word is an instance of a pos
    parseWord(word): Constituent list
        returns the list of all Constituents that can be formed containing this word, according to lexicon
    parseConstituents(lefts, rights): Constituent list
        returns the list of all Constituents that can be formed using any pairing of a Constituent in lefts 
            and a Constituent in rights, according to binGrammar or uniGrammar
    parse(phrase): Constituent set
        returns the set of all Constituents that span the entire phrase
    """
    
    def __init__(self):
        """
        Constructs all neccesary attributes for the Grammar object
        
        Parameters
        N/A
        """
        self.uniGrammar = {}
        self.binGrammar = {}
        self.lexicon = {}
       
    def add(self, i, o):
        """
        Simplified way to enter grammar rules, sorts grammar rules by content to use addBin(), addUni(), or addLexicon()
        
        Parameters
        i : str, POS tuple, or POS tuple tuple
            the "in" of the production rule - the constituent, constituent(s) or word that are subsumed into the Constituent
        o : POS tuple
            the "out" of the production rule - the type of Constituent that is formed
            
        Returns
        None
        """
        if isinstance(i, str):
            if not isinstance(o, set):
                o = [o]
            self.addLexicon(i, o)
        elif isinstance(i, tuple):
            if [type(e) for e in i] == [tuple, tuple]:
                self.addBin(i, o)
            else:
                self.addUni(i, o)
                
    def addBin(self, constituents, phrase):
        """
        Adds a binary production rule to self.binGrammar in which the constituents join together to form a phrase
        
        Parameters
        constituents : POS tuple tuple
            the pair of Constituents (POS tuples) that can be joined into one phrase
        phrase : POS tuple
            the pos of the new Constituent that is formed
                
        Returns
        None
        """
        self.binGrammar[constituents] = phrase
        
    def addUni(self, constituent, phrase):
        """
        Adds a unary production rule to self.uniGrammar in which the constituent forms a phrase
        
        Parameters
        constituent : POS tuple
            the type of Constituent that forms a phrase
        phrase : POS tuple
            the type of phrase that that Constituent can form
                
        Returns
        None
        """
        self.uniGrammar[constituent] = phrase
        
    def addLexicon(self, word, pos):
        """
        Adds an entry to the lexicon in which the word is an instance of a pos
        
        Parameters
        word : str
            the word that is defined as an instance of a particular part of speech
        pos : POS tuple
            the definition of the part of speech that the word is classified as
            
        Returns
        None
        """
        if word.lower() not in self.lexicon:
            self.lexicon[word.lower()] = set()
        for p in pos:
            self.lexicon[word.lower()].add(p)
    
    def parseWord(self, word):
        """
        The list of all Constituents that can be formed containing this word, according to lexicon

        Parameters
        word : str
            the word to be parsed into a leaf Constituent
        
        Returns
        parses (Constituent list):
            the list of all possible Constituents that can be formed containing only this word, according to self.lexicon
        """
        parses = []
        typeList = None
        if word.lower() in self.lexicon:
            typeList = self.lexicon[word.lower()]
        if typeList != None:
            for c in typeList:
                parses.append(Constituent(pos = c, word = word));
        return parses
    
    def parseConstituents(self, lefts, rights):
        """
        Returns the list of all Constituents that can be formed using any pairing of a Constituent in lefts 
            and a Constituent in rights, according to binGrammar or uniGrammar

        Parameters
        lefts : Constituent list
            a list of all Constituents that could become the left child of this Constituent, if the grammar allows
        rights : Constituent list
            a list of all Constituents that could become the right child of this Consituent, if the grammar allows
            
        Returns
            parses (Constituent list):
                the list of all possible Constituents that can be formed using one of the Constituents contained in lefts
                    as a left child and one of the constituents in rights as the right child
        """
        parses = []
        if lefts == rights:
            for left in lefts:
                c = None
                if left.pos in self.uniGrammar:
                    c = self.uniGrammar[left.pos]
                if c != None:
                    
                    constituent = Constituent(pos = c, left = left, right = left)
                    parses.append(constituent)
        else:
            for left in lefts:
                for right in rights:
                    c = None
                    if (left.pos, right.pos) in self.binGrammar:
                        c = self.binGrammar[(left.pos, right.pos)]
                    if c != None:
                        constituent = Constituent(pos = c, left = left, right = right)
                        parses.append(constituent)
        return parses
       
    def parse(self, phrase):
        """
        Parses a phrase into a full tree of syntactic Constituents
        
        Parameters
        phrase: str
            the phrase to parse
        
        Returns
        parses:
            the set of all parses spanning the entire input phrase
        """
        words = re.sub(r'[-;,:\"\'\\)(%$!.?~@^*]', "", phrase).split()
    
        matrix = [[set() for x in range(len(words))] for y in range(len(words))]
    
        for i in range(len(words)):
            y = 0
            x = i
            for j in range(i, len(words)):
                if x == y:
                    word = words[x]
                    for c in self.parseWord(word):
                        matrix[y][x].add(c)
                totalOffset = x - y
                for offset in range(totalOffset):
                    for c in self.parseConstituents(matrix[y][x-totalOffset+offset], matrix[y+offset+1][x]):
                        matrix[y][x].add(c)
                #account for unary production rules
                parseCount = 0#there were 0 or more possible parses
                while len(matrix[y][x]) != parseCount:
                    for c in self.parseConstituents(matrix[y][x], matrix[y][x]):
                        matrix[y][x].add(c)
                    parseCount = len(matrix[y][x])
                #increment
                x += 1
                y += 1
        return matrix[0][len(words)-1]

In [4]:
class Grammars:
    """
    This class contains several example Grammars for demonstration purposes, although users can construct custom Grammars 
    
    Attributes
    N/A
    
    Methods
    simple(): Grammar
        returns a simple Grammar object with three vocabulary words and two production rules
        
    demo(): Grammar
        returns a grammar that only supports pronouns or Determiners + Nouns as noun phrases, and only supports transitive verbs
        used for demonstrating these classes
        
    complex(): Grammar
        returns an overly complex grammar that should very detailedly reflect english syntax, but has errors
    """
    @staticmethod
    def simple():
        """
        returns a simple Grammar object with three vocabulary words and two production rules
        """
        g = Grammar()
        g.add("this", (POS.N, POS.Phrase))
        g.add("is", (POS.V,))
        g.add("syntax", (POS.N, POS.Phrase))
        g.add(((POS.V,), (POS.N, POS.Phrase)), (POS.V, POS.Phrase))
        g.add(((POS.N, POS.Phrase), (POS.V, POS.Phrase)), (POS.T, POS.Phrase))
        return g
        
    @staticmethod
    def demo():
        """
        returns a grammar that only supports pronouns or Determiners + Nouns as noun phrases, and only supports transitive verbs
        used for demonstrating these classes
        """
        g = Grammar()
        #Pronouns
        g.add("I", (POS.N, POS.Pro))
        g.add("me", (POS.N, POS.Pro))
        g.add("we", (POS.N, POS.Pro))
        g.add("us", (POS.N, POS.Pro))
        g.add("you", (POS.N, POS.Pro))
        g.add("he", (POS.N, POS.Pro))
        g.add("him", (POS.N, POS.Pro))
        g.add("she", (POS.N, POS.Pro))
        g.add("her", (POS.N, POS.Pro))
        g.add("they", (POS.N, POS.Pro))
        g.add("them", (POS.N, POS.Pro))
        g.add((POS.N, POS.Pro), (POS.N, POS.Phrase))
        
        #Nouns
        g.add("man", (POS.N,))
        g.add("binoculars", (POS.N,))
        g.add("dog", (POS.N,))
        g.add("park", (POS.N,))
        g.add(((POS.Det,), (POS.N,)), (POS.N, POS.Phrase))#NP -> Det N
        g.add(((POS.N, POS.Phrase), (POS.P, POS.Phrase)), (POS.N, POS.Phrase))#NP -> NP PP
        
        #Determiners
        g.add("the", (POS.Det,))
        g.add("a", (POS.Det,))
        g.add("an", (POS.Det,))
        g.add("my", (POS.Det,))
        
        #Verbs
        g.add("saw", (POS.V,))
        g.add("see", (POS.V,))
        g.add("have", (POS.V,))
        g.add("found", (POS.V,))
        g.add(((POS.V,), (POS.N, POS.Phrase)), (POS.V, POS.Phrase))#VP -> V NP
        g.add(((POS.V, POS.Phrase), (POS.P, POS.Phrase)), (POS.V, POS.Phrase))#VP -> VP PP
        
        #Prepositions
        g.add("with", (POS.P,))
        g.add("in", (POS.P,))
        g.add(((POS.P,), (POS.N, POS.Phrase)), (POS.P, POS.Phrase))#PP -> P NP
        
        g.add(((POS.N, POS.Phrase), (POS.V, POS.Phrase)), (POS.T, POS.Phrase))
        
        return g
        
    @staticmethod
    def complex():#This one is kinda broken cause I tried to make it too complex. 
        """
        returns an overly complex grammar that should very detailedly reflect english syntax, but has errors
        """
        g = Grammar()
        #lexicon
        g.add("I", (POS.N, POS.Pro))#Pronouns
        g.add("me", (POS.N, POS.Pro))
        g.add("we", (POS.N, POS.Pro))
        g.add("us", (POS.N, POS.Pro))
        g.add("you", (POS.N, POS.Pro))
        g.add("he", (POS.N, POS.Pro))
        g.add("him", (POS.N, POS.Pro))
        g.add("she", (POS.N, POS.Pro))
        g.add("her", (POS.N, POS.Pro))
        g.add("they", (POS.N, POS.Pro))
        g.add("them", (POS.N, POS.Pro))
        g.add("saw", (POS.N, POS.Count))#Count Nouns
        g.add("dog", (POS.N, POS.Count))
        g.add("sky", (POS.N, POS.Count))
        g.add("car", (POS.N, POS.Count))
        g.add("cat", (POS.N, POS.Count))
        g.add("person", (POS.N, POS.Count))
        g.add("saws", (POS.N, POS.Plural))#Plural Nouns
        g.add("dogs", (POS.N, POS.Plural))
        g.add("cars", (POS.N, POS.Plural))
        g.add("cats", (POS.N, POS.Plural))
        g.add("people", (POS.N, POS.Plural))
        g.add("water", (POS.N, POS.Mass))#Mass Nouns
        g.add("go", (POS.V, POS.Intransitive))#Intransitive Verbs
        g.add("saw", (POS.V, POS.Transitive))#Transitive Verbs
        g.add("have", (POS.V, POS.Transitive))
        g.add("to", (POS.P,))#Pronouns
        g.add("through", (POS.P,))
        g.add("from", (POS.P,))
        g.add("with", (POS.P,))
        g.add("the", (POS.Det,))#Determiners
        g.add("an", (POS.Det,))
        g.add("a", (POS.Det,))
        #syntax - unary
        g.add((POS.N, POS.Pro), (POS.Det, POS.Phrase))#DetP -> NPro #Nouns 
        g.add((POS.N, POS.Pro), (POS.N, POS.Phrase))#NP -> NPro
        g.add((POS.N, POS.Proper), (POS.Det, POS.Phrase))#DetP -> NPro
        g.add((POS.N, POS.Proper), (POS.N, POS.Phrase))#NP -> NProper
        g.add((POS.N, POS.Mass), (POS.N, POS.Bar))#N' -> NMass
        g.add((POS.N, POS.Plural), (POS.N, POS.Bar))#N' -> NPlural
        g.add((POS.N, POS.Phrase), (POS.Det, POS.Phrase))#DP -> NP
        g.add((POS.N, POS.Count), (POS.N, POS.Count, POS.Bar))
        g.add((POS.N, POS.Bar), (POS.N, POS.Phrase))
        g.add((POS.V, POS.Intransitive), (POS.V, POS.Bar))#Verbs
        g.add((POS.V, POS.Bar), (POS.V, POS.Phrase))
        g.add((POS.Adj,), (POS.Adj, POS.Bar))#Adjectives
        g.add((POS.Adj, POS.Bar), (POS.Adj, POS.Phrase))
        g.add((POS.Adv,), (POS.Adv, POS.Bar))#Adverb
        g.add((POS.Adv, POS.Bar), (POS.Adv, POS.Phrase))
        g.add((POS.Det, POS.Bar), (POS.Det, POS.Phrase))#Determiners
        g.add((POS.P, POS.Bar), (POS.P, POS.Phrase))#Prepositions
        #syntax - binary
        g.add(((POS.Det, POS.Phrase), (POS.V, POS.Phrase)), (POS.T, POS.Phrase))#Sentence
        g.add(((POS.Adv, POS.Phrase), (POS.Adj)), (POS.Adj, POS.Bar))#AdvP + Adj -> Adj' 
        g.add(((POS.Adv, POS.Phrase), (POS.Adv)), (POS.Adv, POS.Bar))#AdvP + Adv -> Adv' 
        g.add(((POS.Adj, POS.Phrase), (POS.N, POS.Count, POS.Bar)), (POS.N, POS.Count, POS.Bar)) #Nouns
        g.add(((POS.N, POS.Count, POS.Bar), (POS.P, POS.Phrase)), (POS.N, POS.Count, POS.Bar))
        g.add(((POS.Adj, POS.Phrase), (POS.N, POS.Bar)), (POS.N, POS.Bar))
        g.add(((POS.N, POS.Bar), (POS.P, POS.Phrase)), (POS.N, POS.Bar))
        g.add(((POS.V, POS.Transitive), (POS.Det, POS.Phrase)), (POS.V, POS.Bar))#Verbs
        g.add(((POS.V, POS.Bar), (POS.Adv, POS.Phrase)), (POS.V, POS.Bar))
        g.add(((POS.V, POS.Bar), (POS.P, POS.Phrase)), (POS.V, POS.Bar))
        g.add(((POS.Det,), (POS.N, POS.Phrase)), (POS.Det, POS.Bar))#Determiners
        g.add(((POS.P,), (POS.Det, POS.Phrase)), (POS.P, POS.Bar))#Prepositions
        return g

# Demonstration

In [5]:
#This is my CYK parser. Let's parse a simple sentence with my simple grammar:
print(Grammars.simple().parse("This is syntax"))

{[TP [NP This] [VP [V is] [NP syntax]]]}


In [6]:
#My parser can handle ambiguity! If there are multiple grammatical parses, it returns both in a python set:
print(Grammars.demo().parse("I saw the man with the binoculars"))

{[TP [NP [N I]] [VP [VP [V saw] [NP [Det the] [N man]]] [PP [P with] [NP [Det the] [N binoculars]]]]], [TP [NP [N I]] [VP [V saw] [NP [NP [Det the] [N man]] [PP [P with] [NP [Det the] [N binoculars]]]]]]}


In [7]:
#(It doesn't have any knowledge of Semantics though, so sometimes it thinks non-ambiguous sentences are actually ambiguous):
print(Grammars.demo().parse("I found my dog in the park"))

{[TP [NP [N I]] [VP [V found] [NP [NP [Det my] [N dog]] [PP [P in] [NP [Det the] [N park]]]]]], [TP [NP [N I]] [VP [VP [V found] [NP [Det my] [N dog]]] [PP [P in] [NP [Det the] [N park]]]]]}


In [8]:
#If it encounters any words it does not know, OR if it knows the words, but they don't
# make sense syntactically, it returns an empty set:
print(Grammars.simple().parse("This is phonology"))
print(Grammars.simple().parse("Syntax this is"))

set()
set()


In [9]:
#It uses regular expressions to ignore punctuation and capitalization (but keeps the 
# original capitalization in the final bracket notation):
print(Grammars.demo().parse("~~~I sA@w;; tH.E b!inoCU::laRs~~~"))

{[TP [NP [N I]] [VP [V sAw] [NP [Det tHE] [N binoCUlaRs]]]]}


In [10]:
#I didn't limit my parser to only display full parses that resolve to a complete sentence:
print(Grammars.demo().parse("with my dog"))
#Instead it displays any parses that span the whole input

{[PP [P with] [NP [Det my] [N dog]]]}


In [11]:
#In addition to using the simple pre-made grammars that I built (in the Grammars class), you can also make custom ones!
deutscheGrammatik = Grammar()
deutscheGrammatik.add("Ich", (POS.N, POS.Phrase)) # NP -> "Ich"
deutscheGrammatik.add("verstehe", (POS.V,)) # V -> "verstehe"
deutscheGrammatik.add("Deutsch", (POS.N, POS.Phrase)) #NP -> "Deutsch"
deutscheGrammatik.add("auch", (POS.Adv,))
deutscheGrammatik.add((POS.Adv,), (POS.Adv, POS.Phrase))
deutscheGrammatik.add(((POS.V,), (POS.N, POS.Phrase)), (POS.V, POS.Phrase)) # VP -> V NP
deutscheGrammatik.add(((POS.V, POS.Phrase), (POS.Adv, POS.Phrase)), (POS.V, POS.Phrase)) # VP -> VP AdvP
deutscheGrammatik.add(((POS.N, POS.Phrase), (POS.V, POS.Phrase)), (POS.T, POS.Phrase)) # TP -> NP VP
print(deutscheGrammatik.parse("Ich verstehe Deutsch auch!"))

{[TP [NP Ich] [VP [VP [V verstehe] [NP Deutsch]] [AdvP [Adv auch]]]]}


In [12]:
#There are three formats to keep in mind when adding rules to a custom grammar:
customGrammar = Grammar()

#vocabulary - use the word as your first parameters, and the Part of Speech (as a tuple) as the second:
customGrammar.add(   "this",   (POS.N, POS.Pro)   ) #"this" is a pronoun
print(customGrammar.parse("this"))
#you can use multiple parses for the same vocabulary word:
customGrammar.add(   "this",     (POS.Det,)   ) #"this" is a determiner
print(customGrammar.parse("this"))

#unary production rules - use a Part of Speech (as a tuple) for both parameters:
customGrammar.add(   (POS.N, POS.Pro),   (POS.N, POS.Phrase)   ) #NP -> Pronoun
print(customGrammar.parse("this"))

#binary production rules - use a tuple of Parts of Speech (both as tuples themselves) for the first parameter, and 
# another Part of Speech (still a tuple) as the second:
customGrammar.add(   (  (POS.Det,),  (POS.N,)  ),   (POS.N, POS.Phrase)   ) #NP -> Det N
customGrammar.add(   (  (POS.N, POS.Phrase),  (POS.V, POS.Phrase)  ),   (POS.T, POS.Phrase)   ) #TP -> NP VP
#adding more grammar/vocabulary to build phrases out of:
customGrammar.add(   "works",     (POS.V,)   ) #"works" is a Verb
customGrammar.add(   (POS.V,),   (POS.V, POS.Phrase)   ) #VP -> V
customGrammar.add(   "work",     (POS.N,)   ) #"works" is a Verb
print(customGrammar.parse("This works!"))
print(customGrammar.parse("This work"))

{[N this]}
{[Det this], [N this]}
{[Det this], [N this], [NP [N this]]}
{[TP [NP [N This]] [VP [V works]]]}
{[NP [Det This] [N work]]}
